In [54]:
import libs.fs
import parse
import numpy as np
import pandas as pd
import io
import scipy.interpolate
import matplotlib.pyplot as plt

In [89]:
TEMPLATE = """RepRapFirmware height map file v2, min error {min:.3f}, max error {max:.3f}, mean {mean:.3f}, deviation {std:.3f}
axis0,axis1,min0,max0,min1,max1,radius,spacing0,spacing1,num0,num1
X,Y,-170.00,169.00,-170.00,170.00,-1.00,56.49,56.66,7,7
{data}
"""

In [90]:
filenames = [
    "/Users/igor/Downloads/heightmaps/heightmap_023C.csv",
    "/Users/igor/Downloads/heightmaps/heightmap_060C.csv",
    "/Users/igor/Downloads/heightmaps/heightmap_100C.csv",
]

In [91]:
def load(filename):
    with open(filename) as f:
        content = f.read()

    parsed = parse.parse(TEMPLATE, content)
    temp = int(filename[-8:-5])
    values = pd.read_csv(io.StringIO(parsed["data"]), header=None).values
    return temp, values

In [92]:
all_temps = [load(filename) for filename in filenames]

In [93]:
temps, values = zip(*all_temps)
temps = np.array(temps)
values = np.array(values)
print(temps.shape, values.shape)
interp = scipy.interpolate.interp1d(temps, values, axis=0, fill_value="extrapolate")

(3,) (3, 7, 7)


In [94]:
!cat /Users/igor/Downloads/heightmaps/heightmap_023C.csv

RepRapFirmware height map file v2, min error -0.030, max error 0.929, mean 0.508, deviation 0.240
axis0,axis1,min0,max0,min1,max1,radius,spacing0,spacing1,num0,num1
X,Y,-170.00,169.00,-170.00,170.00,-1.00,56.49,56.66,7,7
  0.532,  0.447,  0.287,  0.207,  0.248,  0.354,  0.525
  0.402,  0.558,  0.582,  0.540,  0.511,  0.464,  0.331
  0.395,  0.738,  0.882,  0.856,  0.810,  0.620,  0.322
  0.366,  0.765,  0.911,  0.929,  0.849,  0.639,  0.250
  0.423,  0.709,  0.861,  0.850,  0.772,  0.586,  0.331
  0.515,  0.577,  0.605,  0.590,  0.545,  0.532,  0.459
  0.417,  0.200,  0.045, -0.015, -0.030,  0.136,  0.436


In [109]:
def write_interp(t):
    y = interp(t)
    data_rows = [
        "".join(f"  {val:.3f}," for val in row)[:-1]
        for row in y
    ]
    data_csv = "\n".join(data_rows)
    content = TEMPLATE.format(
        min=np.min(y),
        max=np.max(y),
        mean=np.mean(y),
        std=np.std(y),
        data=data_csv
    )
    with open("/Users/igor/Downloads/heightmaps/heightmap_{t}C_interp.csv".format(t=str(t).zfill(3)), "wt") as f:
        f.write(content)

In [110]:
for t in [20, 30, 40, 50, 60, 70, 80, 90, 100]:
    write_interp(t)